# Keypoint Detection using YOLO v7 #

Decision points:
    - transfer learning by weigths vs. new weights

In [ ]:
# import packages and other files
import torch
assert torch.__version__.startswith("1.8") 
import torchvision
import cv2
import os
import numpy as np
import json
import random
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import sys
from utils.datasets import letterbox
from utils.general import non_max_suppression_kpt
from utils.plots import output_to_keypoint, plot_skeleton_kpts

In [ ]:
# Reading in images and annotations

directory = "Documents/Studium/- UiA 2023SS/Master Research Project/Smordalen_M1_2022-04-11_biomass_correct/images"
classes = ["fish"]

def get_data_dicts(directory, classes):
    dataset_dicts = []
    cnt=0
    for filename in [file for file in os.listdir(directory) if file.endswith('.json')]:
        json_file = os.path.join(directory, filename)
        with open(json_file) as f:
            img_anns = json.load(f)
        record = {}
        filename = os.path.join(directory, img_anns["imagePath"])
        record["file_name"] = filename
        ########################check your image and replace this two
        record["height"] = 736       ##########image size
        record["width"] = 1920  
        record["image_id"]=cnt  
        #record["name"]="fish"
        
        #objjs = []
        #objj = {"id": cnt,"file_name": filename,"height": 736,"width": 1920}
        #objjs.append(objj)
        #record["images"] = objjs

        annos = img_anns["shapes"]
        objs = []
        pxx=[]
        pyy=[]
        #print('annos 2 is',annos[2])
        iii=0
        for anno in annos:
          if(iii==0):
            px = [a[0] for a in anno['points']] # x coord
            #print('px vlues are',px)
            py = [a[1] for a in anno['points']] # y-coord
            poly = [(x, y) for x, y in zip(px, py)] # poly for segmentation
            poly = [p for x in poly for p in x]
            #print('for i',iii)
            iii=iii+1
          else:
            iii=iii+1  
            if(iii>5):  ###########orignal 5
              pxi = [a[0] for a in anno['points']] # x coord
              pyi = [a[1] for a in anno['points']] # y-coord
              pt = [(x, y) for x, y in zip(pxi, pyi)] # poly for segmentation
              pt = [p for x in pt for p in x]
              pxx.append(pt[0])
              #print('px array values are for y',pt[1])
              #print('px vlues are',pxx)
              pyy.append(pt[1])
              #print('for i',iii)
        bx=[np.min(px), np.min(py), np.max(px), np.max(py)]      
        key_pt=[pxx[0],pyy[0],2,pxx[1],pyy[1],2,pxx[2],pyy[2],2,pxx[3],pyy[3],2,pxx[4],pyy[4],2,pxx[5],pyy[5],2,pxx[6],pyy[6],2]
        #key_pt=[100,200,2,150,250,2,50,50,2,200,200,2,450,250,2,50,50,2,350,350,2]
        print('bx is',bx,'keypoint is',key_pt)
        obj = {
                "bbox": [np.min(px), np.min(py), np.max(px), np.max(py)],
                "bbox_mode": BoxMode.XYXY_ABS,
                "segmentation": [poly],
                "keypoints":[key_pt],
                "id":cnt,
                "category_id":0, #classes.index(anno['label']),
                "iscrowd": 0,
                "num_keypoints":7
              }
        objs.append(obj)
        record["annotations"] = objs
        print('record is',record)
        dataset_dicts.append(record)
        cnt=cnt+1
    return dataset_dicts

print("Done creating dataset dict")

In [ ]:
# YOLO v7 setup

